In [1]:
import pandas as pd
import numpy as np

from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import OrdinalEncoder

## Загрузка данных и предобработка

In [2]:
data = pd.read_excel('/content/задание.xlsx',sheet_name='факт')
data2 = pd.read_excel('/content/задание.xlsx',sheet_name='тест')

In [3]:
display(data.info())
display(data2.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9500 entries, 0 to 9499
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   ФИО       9500 non-null   object
 1   ПОЛ       9500 non-null   object
 2   Документ  9157 non-null   object
 3   Страна    9500 non-null   object
dtypes: object(4)
memory usage: 297.0+ KB


None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2500 entries, 0 to 2499
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   ФИО       2500 non-null   object 
 1   ПОЛ       0 non-null      float64
 2   Документ  2413 non-null   object 
 3   Страна    0 non-null      float64
dtypes: float64(2), object(2)
memory usage: 78.2+ KB


None

In [4]:
display(data.head())
display(data2.head())

,ФИО,ПОЛ,Документ,Страна
0,АБАДЖЯН МКРТИЧ МКРТЫЧЕВИЧ,М,AP!!!!!!,АРМЕНИЯ
1,АБАДУЛЛАЕВ РАШИДБАЙ КАНДАХАРОВИЧ,М,AB!!!!!!!,УЗБЕКИСТАН
2,АБАЕВ ЙУЛДОШ ЮЛДОШОВИЧ,М,СР!!!!!!!,УЗБЕКИСТАН
3,АБАЗЯН ГЕВОРГ ЕНОКОВИЧ,М,AP!!!!!!,АРМЕНИЯ
4,АБАЙ УУЛУ ДАНИЯРБЕК МИХАЙЛОВИЧ,М,NaN,КИРГИЗИЯ


,ФИО,ПОЛ,Документ,Страна
0,АБАСОВ РАШАД РАДЖАБ ОГЛЫ,NaN,С!!!!!!!,NaN
1,АБАСОВА УЛЬЯНА ИГОРЕВНА,NaN,!!!!!!!!!!,NaN
2,АББАСОВ ДЖЕЙХУН ДАВУД ОГЛЫ,NaN,!!!!!!!!!!,NaN
3,АББАСОВ ДУНЯМИН МАХМУД ОГЛЫ,NaN,З!!!!!!!,NaN
4,АББАСОВ ЕЛЬНУР АРИЗ ОГЛЫ,NaN,С!!!!!!!,NaN


In [5]:
display(data.isna().sum())
display(data2.isna().sum())

ФИО           0
ПОЛ           0
Документ    343
Страна        0
dtype: int64

ФИО            0
ПОЛ         2500
Документ      87
Страна      2500
dtype: int64

In [6]:
data['Документ'] = data['Документ'].fillna(0).astype(str)
data2['Документ'] = data2['Документ'].fillna(0).astype(str)

In [7]:
data2 = data2.drop(['ПОЛ', 'Страна'], axis=1)

In [8]:
coder_genre = OrdinalEncoder()
data['target_1'] = coder_genre.fit_transform(data[['ПОЛ']])
data.head(10)

,ФИО,ПОЛ,Документ,Страна,target_1
0,АБАДЖЯН МКРТИЧ МКРТЫЧЕВИЧ,М,AP!!!!!!,АРМЕНИЯ,1.0
1,АБАДУЛЛАЕВ РАШИДБАЙ КАНДАХАРОВИЧ,М,AB!!!!!!!,УЗБЕКИСТАН,1.0
2,АБАЕВ ЙУЛДОШ ЮЛДОШОВИЧ,М,СР!!!!!!!,УЗБЕКИСТАН,1.0
3,АБАЗЯН ГЕВОРГ ЕНОКОВИЧ,М,AP!!!!!!,АРМЕНИЯ,1.0
4,АБАЙ УУЛУ ДАНИЯРБЕК МИХАЙЛОВИЧ,М,0,КИРГИЗИЯ,1.0
5,АБАЙ УУЛУ НУРСУЛТАН МАХАМАДСИДИКОВИЧ,М,!!!!!!!,КИРГИЗИЯ,1.0
6,АБАКАРОВ МУХАММАД МИХАЙЛОВИЧ,М,!!!!!!!!!!,РОССИЯ,1.0
7,АБАРАЛИЕВ КАМИЛЬ,М,АС!!!!!!!,КИРГИЗИЯ,1.0
8,АБАРАЛИЕВ РАФИКЖОН,М,АС!!!!!!!,КИРГИЗИЯ,1.0
9,АБАСЗАДЕ ГАМДУЛЛА РАФИК ОГЛЫ,М,!!!!!!!!!,АЗЕРБАЙДЖАН,1.0


In [9]:
coder_country = OrdinalEncoder()
data['target_2'] = coder_country.fit_transform(data[['Страна']])
data.head(10)

,ФИО,ПОЛ,Документ,Страна,target_1,target_2
0,АБАДЖЯН МКРТИЧ МКРТЫЧЕВИЧ,М,AP!!!!!!,АРМЕНИЯ,1.0,1.0
1,АБАДУЛЛАЕВ РАШИДБАЙ КАНДАХАРОВИЧ,М,AB!!!!!!!,УЗБЕКИСТАН,1.0,4.0
2,АБАЕВ ЙУЛДОШ ЮЛДОШОВИЧ,М,СР!!!!!!!,УЗБЕКИСТАН,1.0,4.0
3,АБАЗЯН ГЕВОРГ ЕНОКОВИЧ,М,AP!!!!!!,АРМЕНИЯ,1.0,1.0
4,АБАЙ УУЛУ ДАНИЯРБЕК МИХАЙЛОВИЧ,М,0,КИРГИЗИЯ,1.0,2.0
5,АБАЙ УУЛУ НУРСУЛТАН МАХАМАДСИДИКОВИЧ,М,!!!!!!!,КИРГИЗИЯ,1.0,2.0
6,АБАКАРОВ МУХАММАД МИХАЙЛОВИЧ,М,!!!!!!!!!!,РОССИЯ,1.0,3.0
7,АБАРАЛИЕВ КАМИЛЬ,М,АС!!!!!!!,КИРГИЗИЯ,1.0,2.0
8,АБАРАЛИЕВ РАФИКЖОН,М,АС!!!!!!!,КИРГИЗИЯ,1.0,2.0
9,АБАСЗАДЕ ГАМДУЛЛА РАФИК ОГЛЫ,М,!!!!!!!!!,АЗЕРБАЙДЖАН,1.0,0.0


In [10]:
coder_doc = OrdinalEncoder(handle_unknown= 'use_encoded_value', unknown_value=-1)
data['features_doc'] = coder_doc.fit_transform(data[['Документ']])
data2['features_doc'] = coder_doc.transform(data2[['Документ']])
display(data.head(10))
display(data2.head(10))

,ФИО,ПОЛ,Документ,Страна,target_1,target_2,features_doc
0,АБАДЖЯН МКРТИЧ МКРТЫЧЕВИЧ,М,AP!!!!!!,АРМЕНИЯ,1.0,1.0,53.0
1,АБАДУЛЛАЕВ РАШИДБАЙ КАНДАХАРОВИЧ,М,AB!!!!!!!,УЗБЕКИСТАН,1.0,4.0,32.0
2,АБАЕВ ЙУЛДОШ ЮЛДОШОВИЧ,М,СР!!!!!!!,УЗБЕКИСТАН,1.0,4.0,497.0
3,АБАЗЯН ГЕВОРГ ЕНОКОВИЧ,М,AP!!!!!!,АРМЕНИЯ,1.0,1.0,53.0
4,АБАЙ УУЛУ ДАНИЯРБЕК МИХАЙЛОВИЧ,М,0,КИРГИЗИЯ,1.0,2.0,26.0
5,АБАЙ УУЛУ НУРСУЛТАН МАХАМАДСИДИКОВИЧ,М,!!!!!!!,КИРГИЗИЯ,1.0,2.0,1.0
6,АБАКАРОВ МУХАММАД МИХАЙЛОВИЧ,М,!!!!!!!!!!,РОССИЯ,1.0,3.0,4.0
7,АБАРАЛИЕВ КАМИЛЬ,М,АС!!!!!!!,КИРГИЗИЯ,1.0,2.0,365.0
8,АБАРАЛИЕВ РАФИКЖОН,М,АС!!!!!!!,КИРГИЗИЯ,1.0,2.0,365.0
9,АБАСЗАДЕ ГАМДУЛЛА РАФИК ОГЛЫ,М,!!!!!!!!!,АЗЕРБАЙДЖАН,1.0,0.0,3.0


,ФИО,Документ,features_doc
0,АБАСОВ РАШАД РАДЖАБ ОГЛЫ,С!!!!!!!,474.0
1,АБАСОВА УЛЬЯНА ИГОРЕВНА,!!!!!!!!!!,4.0
2,АББАСОВ ДЖЕЙХУН ДАВУД ОГЛЫ,!!!!!!!!!!,4.0
3,АББАСОВ ДУНЯМИН МАХМУД ОГЛЫ,З!!!!!!!,-1.0
4,АББАСОВ ЕЛЬНУР АРИЗ ОГЛЫ,С!!!!!!!,474.0
5,АББАСОВ РАФИГ РАШИД ОГЛЫ ЗАКИР ОГЛЫ,!!!!!!!!!!,4.0
6,АББАСОВ РОВШАН МУСАДДИН ОГЛЫ,С!!!!!!,473.0
7,АББАСОВ ФИРУДДИН АДАЛАТ ОГЛЫ,P!!!!!!!,267.0
8,АББАСОВ ЭЛЬФАГ ИБРАГИМ ОГЛЫ,P!!!!!!!!,268.0
9,АББАСОВА МЕРИЛИН РАХИБ КЫЗЫ,!!!!!!,0.0


## Обучение модели по определению пола

In [11]:
features = data['ФИО']
target = data['target_1']
print(features.shape)
print(target.shape)

(9500,)
(9500,)


In [12]:
tf_idf = TfidfVectorizer()

In [13]:
tree_mod = DecisionTreeClassifier(random_state=123)
forest_mod = RandomForestClassifier(random_state=123)
log_mod = LogisticRegression(random_state=123, class_weight='balanced')


step_tree = [('normalizer', tf_idf), ('classifier', tree_mod)]
step_forest = [('normalizer', tf_idf), ('classifier', forest_mod)]
step_log = [('normalizer', tf_idf), ('classifier', log_mod)]

pr_tree = {     
   'classifier__max_depth':list(range(1,15,5)) 
}
pr_forest = {
    'classifier__max_depth': list(range(5,15,5)),
    'classifier__n_estimators': [100, 200],
}
pr_log = {
    'classifier__solver': ['newton-cg'],
    'classifier__max_iter': [100, 150]
}

In [14]:
pipe = Pipeline(step_forest)
forest_gs = GridSearchCV(
    estimator=pipe, scoring='f1', param_grid=pr_forest, cv=3, n_jobs=-1, verbose=True).fit(features, target)

print(forest_gs.best_score_)
print(forest_gs.best_params_)

Fitting 3 folds for each of 4 candidates, totalling 12 fits
0.8051327261306125
{'classifier__max_depth': 10, 'classifier__n_estimators': 100}


In [15]:
pipe = Pipeline(step_log)
log_gs = GridSearchCV(
    estimator=pipe, scoring='f1', param_grid=pr_log, cv=3, n_jobs=-1, verbose=True).fit(features, target)

print(log_gs.best_params_)
print(log_gs.best_score_)

Fitting 3 folds for each of 2 candidates, totalling 6 fits
{'classifier__max_iter': 100, 'classifier__solver': 'newton-cg'}
0.9672791019187508


In [16]:
pipe = Pipeline(step_tree)
tree_gs = GridSearchCV(
    estimator=pipe, scoring='f1', param_grid=pr_tree, cv=3, n_jobs=-1, verbose=True).fit(features, target)

print(tree_gs.best_score_)
print(tree_gs.best_params_)

Fitting 3 folds for each of 3 candidates, totalling 9 fits
0.8606034394236642
{'classifier__max_depth': 11}


In [17]:
features_test = data2['ФИО']
tf_idf = TfidfVectorizer()
features = tf_idf.fit_transform(features)
features_test = tf_idf.transform(features_test)

print(features.shape)
print(features_test.shape)

(9500, 12083)
(2500, 12083)


In [18]:
model = LogisticRegression(random_state=123, class_weight='balanced', max_iter= 100, solver= 'newton-cg')
model.fit(features, target)
predict = model.predict(features_test)

In [19]:
data2['ПОЛ'] = predict
data2['ПОЛ'] = coder_genre.inverse_transform(data2[['ПОЛ']])
data2.head(15)

,ФИО,Документ,features_doc,ПОЛ
0,АБАСОВ РАШАД РАДЖАБ ОГЛЫ,С!!!!!!!,474.0,М
1,АБАСОВА УЛЬЯНА ИГОРЕВНА,!!!!!!!!!!,4.0,Ж
2,АББАСОВ ДЖЕЙХУН ДАВУД ОГЛЫ,!!!!!!!!!!,4.0,М
3,АББАСОВ ДУНЯМИН МАХМУД ОГЛЫ,З!!!!!!!,-1.0,М
4,АББАСОВ ЕЛЬНУР АРИЗ ОГЛЫ,С!!!!!!!,474.0,М
5,АББАСОВ РАФИГ РАШИД ОГЛЫ ЗАКИР ОГЛЫ,!!!!!!!!!!,4.0,М
6,АББАСОВ РОВШАН МУСАДДИН ОГЛЫ,С!!!!!!,473.0,М
7,АББАСОВ ФИРУДДИН АДАЛАТ ОГЛЫ,P!!!!!!!,267.0,М
8,АББАСОВ ЭЛЬФАГ ИБРАГИМ ОГЛЫ,P!!!!!!!!,268.0,М
9,АББАСОВА МЕРИЛИН РАХИБ КЫЗЫ,!!!!!!,0.0,Ж


### Обучение модели по определению страны

In [20]:
class TFidfDop:

    def __init__(self):
        self.tf = None
  
    def fit(self, X, y=None):
        fio = X['ФИО']
        self.tf = TfidfVectorizer().fit(fio)

    def fit_transform(self, X, y=None):
        fio = X['ФИО']
        self.tf = TfidfVectorizer().fit(fio)
        X_new = self.tf.transform(fio).toarray()
        X_new = np.column_stack((X_new, np.array(X[['features_doc']])))
        return X_new
  
    def transform(self, X, y=None):
        fio = X['ФИО']
        X_new = self.tf.transform(fio).toarray()
        X_new = np.column_stack((X_new, np.array(X[['features_doc']])))
        return X_new

In [21]:
tf_idf = TFidfDop()

In [22]:
features = data[['ФИО', 'features_doc']]
target = data['target_2']
print(features.shape)
print(target.shape)

(9500, 2)
(9500,)


In [23]:
tree_mod = DecisionTreeClassifier(random_state=123)
forest_mod = RandomForestClassifier(random_state=123)
log_mod = LogisticRegression(random_state=123, class_weight='balanced')


step_tree = [('normalizer', tf_idf), ('classifier', tree_mod)]
step_forest = [('normalizer', tf_idf), ('classifier', forest_mod)]
step_log = [('normalizer', tf_idf), ('classifier', log_mod)]

pr_tree = {     
   'classifier__max_depth':list(range(1,15,5)) 
}
pr_forest = {
    'classifier__max_depth': list(range(5,15,5)),
    'classifier__n_estimators': [100, 200],
}
pr_log = {
    'classifier__solver': ['newton-cg', 'liblinear', 'sag'],
    'classifier__max_iter': [100, 150]
}

In [24]:
pipe = Pipeline(step_forest)
forest_gs = GridSearchCV(
    estimator=pipe, scoring='accuracy', param_grid=pr_forest, cv=3, n_jobs=-1, verbose=True).fit(features, target)

print(forest_gs.best_score_)
print(forest_gs.best_params_)

Fitting 3 folds for each of 4 candidates, totalling 12 fits


/usr/local/lib/python3.7/dist-packages/joblib/externals/loky/process_executor.py:705: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


0.6030529219818801
{'classifier__max_depth': 10, 'classifier__n_estimators': 200}


In [25]:
pipe = Pipeline(step_log)
log_gs = GridSearchCV(
    estimator=pipe, scoring='accuracy', param_grid=pr_log, cv=3, n_jobs=-1, verbose=True).fit(features, target)

print(log_gs.best_params_)
print(log_gs.best_score_)

Fitting 3 folds for each of 6 candidates, totalling 18 fits


/usr/local/lib/python3.7/dist-packages/joblib/externals/loky/process_executor.py:705: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


{'classifier__max_iter': 100, 'classifier__solver': 'newton-cg'}
0.7461063545992399


In [26]:
pipe = Pipeline(step_tree)
tree_gs = GridSearchCV(
    estimator=pipe, scoring='accuracy', param_grid=pr_tree, cv=3, n_jobs=-1, verbose=True).fit(features, target)

print(tree_gs.best_score_)
print(tree_gs.best_params_)

Fitting 3 folds for each of 3 candidates, totalling 9 fits
0.7950550538850085
{'classifier__max_depth': 11}


In [27]:
tf_idf = TFidfDop()
features = data[['ФИО', 'features_doc']]
features = tf_idf.fit_transform(features)
features_test = data2[['ФИО', 'features_doc']]
features_test = tf_idf.transform(features_test)
print(features.shape)
print(features_test.shape)

(9500, 12084)
(2500, 12084)


In [28]:
model = DecisionTreeClassifier(random_state=123, max_depth=11)
model.fit(features, target)
predict = model.predict(features_test)

In [30]:
data2['Страна'] = predict
data2['Страна'] = coder_country.inverse_transform(data2[['Страна']])
data2 = data2.drop(['features_doc'], axis=1)
data2.head(15)

,ФИО,Документ,ПОЛ,Страна
0,АБАСОВ РАШАД РАДЖАБ ОГЛЫ,С!!!!!!!,М,АЗЕРБАЙДЖАН
1,АБАСОВА УЛЬЯНА ИГОРЕВНА,!!!!!!!!!!,Ж,РОССИЯ
2,АББАСОВ ДЖЕЙХУН ДАВУД ОГЛЫ,!!!!!!!!!!,М,АЗЕРБАЙДЖАН
3,АББАСОВ ДУНЯМИН МАХМУД ОГЛЫ,З!!!!!!!,М,АЗЕРБАЙДЖАН
4,АББАСОВ ЕЛЬНУР АРИЗ ОГЛЫ,С!!!!!!!,М,АЗЕРБАЙДЖАН
5,АББАСОВ РАФИГ РАШИД ОГЛЫ ЗАКИР ОГЛЫ,!!!!!!!!!!,М,АЗЕРБАЙДЖАН
6,АББАСОВ РОВШАН МУСАДДИН ОГЛЫ,С!!!!!!,М,АЗЕРБАЙДЖАН
7,АББАСОВ ФИРУДДИН АДАЛАТ ОГЛЫ,P!!!!!!!,М,АЗЕРБАЙДЖАН
8,АББАСОВ ЭЛЬФАГ ИБРАГИМ ОГЛЫ,P!!!!!!!!,М,АЗЕРБАЙДЖАН
9,АББАСОВА МЕРИЛИН РАХИБ КЫЗЫ,!!!!!!,Ж,КИРГИЗИЯ


In [31]:
data2.to_excel('response_test.xlsx', index=False)